## Exercise: Wrangling Data: Acquisition, Integration, and Exploration

For this lab’s exercise we are going to answer a few questions about AirBnB listings in San Francisco to make a better informed civic descisions. Spurred by Prop F in San Francisco, imagine you are the mayor of SF (or your respective city) and you need to decide what impact AirBnB has had on your own housing situation. We will collect the relevant data, parse and store this data in a structured form, and use statistics and visualization to both better understand our own city and potentially communicate these findings to the public at large.

> I will explore SF's data, but the techniques should be generally applicable to any city. Inside AirBnB has many interesting cities to further explore: http://insideairbnb.com/

## Outline

* Start with Effective Questions
    * Intro + Data Science Overview
    * Proposition F
    * How can we answer this?
* Acquiring Data
    * What's an API? (Zillow API, SF Open Data, datausa.io)
    * How the Web Works (Socrata API)
* What if there is no API?
    * Scrape an AirBnB listing
* What to do now that we have data?
    * Basics of HTML (CSS selectors and grabbing what you want)
    * Use `lxml` to parse web pages
* Storing Data
    * Schemas and Structure
    * Relations (users, listings, and reviews)
    * Store listing in SQLite
* Manipulating Data
    * basics of Pandas
    * summary stats
    * split-apply-combine
    * Aggregations
    * Prop F. revenue lost
* Exploratory Data Analysis
    * Inside AirBnB
    * Why visual?
    * Chart Types (visualizing continuous, categorical, and distributions and facets)
    * Distributions of Prop F. Revenue vs. point statistics

## Download

In [1]:
import requests

In [2]:
# nice and easy (and structured)
requests.get('https://data.sfgov.org/resource/n5ik-nmm3.json').json()

[{u'best_stat': u'BP Filed',
  u'beststat_group': u'BP Filed',
  u'block_lot': u'0686040',
  u'dbi_filed': u'23-Sep-14',
  u'dbi_permit': u'201409237117',
  u'dbi_project_description': u'CHANGE OF USE: CONVERTING (E) 2ND FLOOR RETAIL SPACE (3,475 SF) TO FOUR RESIDENTIAL APARTMENT UNITS.',
  u'entitlementstatus': u'0',
  u'firstfiled': u'23-Sep-14',
  u'heightnum': u'9999',
  u'landuse': u'Resident',
  u'location_1': {u'human_address': u'{"address":"1656 Post St","city":"","state":"","zip":""}',
   u'latitude': u'37.785772',
   u'longitude': u'-122.428996',
   u'needs_recoding': False},
  u'net_added_sf': u'0',
  u'net_added_units': u'4',
  u'objectid': u'6367',
  u'planning_neighborhood': u'Japantown',
  u'planningdistrictscombo': u'Japantown',
  u'taz': u'705',
  u'total_gsf_commercial': u'0',
  u'totalsqftestim': u'4800',
  u'units': u'4',
  u'zoning': u'NC-2',
  u'zoning_generalized': u'Mixed Use',
  u'zoning_simplified': u'NC-2'},
 {u'best_stat': u'CONSTRUCTION',
  u'beststat_group

In [3]:
# make a naive request for an AirBnB listing
res = requests.get('https://www.airbnb.com/rooms/562311')

# did it work?
res.status_code

503

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'}

res = requests.get('https://www.airbnb.com/rooms/562311', headers=headers)

In [5]:
res.status_code

200

In [6]:
res.content

'<!DOCTYPE html>\n\n<!--[if lt IE 8]>\n\n<html lang="en"\n      \n      xmlns:fb="http://ogp.me/ns/fb#"\n      class="ie">\n\n<![endif]-->\n\n<!--[if IE 8]>\n\n<html lang="en"\n      \n      xmlns:fb="http://ogp.me/ns/fb#"\n      class="ie ie8">\n\n<![endif]-->\n\n<!--[if IE 9]>\n\n  <html lang="en"\n      \n      xmlns:fb="http://ogp.me/ns/fb#"\n      class="ie ie9">\n\n<![endif]-->\n\n<!--[if (gt IE 9)|!(IE)]><!-->\n<html lang="en"\n      \n      xmlns:fb="http://ogp.me/ns/fb#">\n\n<!--<![endif]-->\n\n  <head>\n    <script>\n\n      var sherlock_firstbyte = Number(new Date());\n</script>\n      <link rel="dns-prefetch" href="//maps.googleapis.com">\n      <link rel="dns-prefetch" href="//maps.gstatic.com">\n      <link rel="dns-prefetch" href="//mts0.googleapis.com">\n      <link rel="dns-prefetch" href="//mts1.googleapis.com">\n\n    <!--[if IE]><![endif]-->\n    <meta charset="utf-8">\n\n    <!--[if IE 8]>\n      <link href="https://a2.muscache.com/airbnb/static/packages/common_o2.

In [7]:
# always store raw data. Separate downloading and parsing
with open('airbnb_listing.html', 'w') as fh:
    fh.write(res.content)

In [8]:
!open airbnb_listing.html

## Parse

In [9]:
import lxml.html

# build the DOM Tree
tree = lxml.html.fromstring(res.content)

# print the parsed DOM Tree
print lxml.html.tostring(tree)

<html lang="en" xmlns:fb="http://ogp.me/ns/fb#">

<!--<![endif]-->

  <head>
    <script>

      var sherlock_firstbyte = Number(new Date());
</script>
      <link rel="dns-prefetch" href="//maps.googleapis.com">
      <link rel="dns-prefetch" href="//maps.gstatic.com">
      <link rel="dns-prefetch" href="//mts0.googleapis.com">
      <link rel="dns-prefetch" href="//mts1.googleapis.com">

    <!--[if IE]><![endif]-->
    <meta charset="utf-8">

    <!--[if IE 8]>
      <link href="https://a2.muscache.com/airbnb/static/packages/common_o2.1_ie8-4664dbaa19b79cfe89dfd3a4ddc8a619.css" media="all" rel="stylesheet" type="text/css" />
    <![endif]-->
    <!--[if !(IE 8)]><!-->
      <link href="https://a1.muscache.com/airbnb/static/packages/common_o2.1-bfd1b12b201c6fd210e29e257ff43ba2.css" media="all" rel="stylesheet" type="text/css">
    <!--<![endif]-->

    <!--[if lt IE 9]>
      <link href="https://a0.muscache.com/airbnb/static/compatibility/airglyphs-ie8-abb6df6c7f1568ba147fd36893144e

In [10]:
results = tree.cssselect('div.star-rating')
print results

# print the HTML for the first result.
match = results[0]
print lxml.html.tostring(match)

# Apply the selector to the DOM tree.
results = tree.cssselect('.summary-component div.star-rating')
print results

# print the HTML for the first result.
match = results[0]
print lxml.html.tostring(match)

[<Element div at 0x10c94ed60>, <Element div at 0x10c94edb8>, <Element div at 0x10c94ee10>, <Element div at 0x10c94ee68>, <Element div at 0x10c94eec0>, <Element div at 0x10c94ef18>, <Element div at 0x10c94ef70>, <Element div at 0x10c94efc8>, <Element div at 0x10c7e2050>]
<div class="star-rating">
        <i class="icon icon-beach icon-star"></i>
        <i class="icon icon-beach icon-star"></i>
        <i class="icon icon-beach icon-star"></i>
        <i class="icon icon-beach icon-star"></i>
        <i class="icon icon-star icon-light-gray"></i>
      </div>
      
[<Element div at 0x10c94edb8>]
<div class="star-rating" itemprop="ratingValue" content="5" data-reactid=".4inddseqyo.0.0.1.2.2.0.0"><div class="foreground" data-reactid=".4inddseqyo.0.0.1.2.2.0.0.0"><span data-reactid=".4inddseqyo.0.0.1.2.2.0.0.0.0"><span data-reactid=".4inddseqyo.0.0.1.2.2.0.0.0.0.0:$0"><i class="icon-star icon icon-beach star-rating-icons" data-reactid=".4inddseqyo.0.0.1.2.2.0.0.0.0.0:$0.0"></i><span data-

In [19]:
# Get user profile link
host_link = tree.cssselect('#host-profile a.link-reset')

print lxml.html.tostring(host_link[0])

# Follow User Profile link
host_url = host_link[0].attrib['href']
print host_url

profile = requests.get('http://airbnb.com' + host_url, headers = headers)

<a class="link-reset" href="/users/show/2694430" data-reactid=".uja3tlescg.2.0.0.0.2.1.0.0">Angie</a>
/users/show/2694430


In [20]:
profile.text

u'<!DOCTYPE html>\n\n<!--[if lt IE 8]>\n\n<html lang="en"\n      \n      xmlns:fb="http://ogp.me/ns/fb#"\n      class="ie">\n\n<![endif]-->\n\n<!--[if IE 8]>\n\n<html lang="en"\n      \n      xmlns:fb="http://ogp.me/ns/fb#"\n      class="ie ie8">\n\n<![endif]-->\n\n<!--[if IE 9]>\n\n  <html lang="en"\n      \n      xmlns:fb="http://ogp.me/ns/fb#"\n      class="ie ie9">\n\n<![endif]-->\n\n<!--[if (gt IE 9)|!(IE)]><!-->\n<html lang="en"\n      \n      xmlns:fb="http://ogp.me/ns/fb#">\n\n<!--<![endif]-->\n\n  <head>\n    <script>\n\n      var sherlock_firstbyte = Number(new Date());\n</script>\n      <link rel="dns-prefetch" href="//maps.googleapis.com">\n      <link rel="dns-prefetch" href="//maps.gstatic.com">\n      <link rel="dns-prefetch" href="//mts0.googleapis.com">\n      <link rel="dns-prefetch" href="//mts1.googleapis.com">\n\n    <!--[if IE]><![endif]-->\n    <meta charset="utf-8">\n\n    <!--[if IE 8]>\n      <link href="https://a2.muscache.com/airbnb/static/packages/common_o2

# Query

In [22]:
import pandas as pd

# We will use the Inside AirBnB dataset from here on
df = pd.read_csv('data/sf_listings.csv')
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,1162609,Lovely One Bedroom Apartment,6368122,Taylor,NaN,Seacliff,37.785217,-122.488655,Entire home/apt,350,4,8,2015-09-17,0.28,1,90
1,6032828,Historic Seacliff Home,30384615,Patricia,NaN,Seacliff,37.783658,-122.489398,Entire home/apt,300,1,0,NaN,NaN,1,173
2,6938818,Best Secret in Town,36381578,Harris,NaN,Seacliff,37.781505,-122.504754,Private room,119,1,10,2015-10-08,2.33,1,341
3,8087607,Single Room Beautiful Beach Condo,3264449,Keith,NaN,Seacliff,37.775318,-122.511621,Private room,79,1,0,NaN,NaN,1,40
4,4781448,3 Bd 2.5 Ba Full Flat Condo w Views,13112558,Pam,NaN,Seacliff,37.781797,-122.492492,Entire home/apt,695,2,1,2015-08-11,0.36,1,23


### Using pandas answer the following questions with summary statistics (just numbers, no charts):

* How many neighborhoods?
* Most common room_type
* Neighborhood with the most # of listings
* User with the most # of listings
* Most expensive neighborhood (median price)
* Listing with most # of reviews

In [23]:
# How Many Neighborhoods
neighborhoods = df.neighbourhood.unique()

print neighborhoods
print "\n"
print "There are {0} neighborhoods in San Francisco".format(len(neighborhoods))

['Seacliff' 'Haight Ashbury' 'Outer Mission' 'Downtown/Civic Center'
 'Diamond Heights' 'Lakeshore' 'Russian Hill' 'Noe Valley' 'Inner Sunset'
 'Treasure Island/YBI' 'Outer Richmond' 'Crocker Amazon' 'Excelsior'
 'Parkside' 'Financial District' 'Ocean View' 'Mission'
 'West of Twin Peaks' 'Inner Richmond' 'Marina' 'Bayview'
 'Visitacion Valley' 'Pacific Heights' 'Presidio Heights' 'South of Market'
 'Glen Park' 'Potrero Hill' 'Castro/Upper Market' 'Twin Peaks'
 'Bernal Heights' 'Chinatown' 'North Beach' 'Presidio' 'Nob Hill'
 'Outer Sunset' 'Western Addition' 'Golden Gate Park']


There are 37 neighborhoods in San Francisco


In [24]:
# Most Common Room Type
listing_types = df.room_type.value_counts()
print listing_types
print "'{0}' is the most common listing type with {1} total listings".format(listing_types.index[0], listing_types[0])

Entire home/apt    4033
Private room       2624
Shared room         372
Name: room_type, dtype: int64
'Entire home/apt' is the most common listing type with 4033 total listings


In [25]:
# Neighborhood with the Most Listings
df.groupby('neighbourhood').count().sort_values('id', ascending=False).iloc[0]

id                                922
name                              922
host_id                           922
host_name                         860
neighbourhood_group                 0
latitude                          922
longitude                         922
room_type                         922
price                             922
minimum_nights                    922
number_of_reviews                 922
last_review                       785
reviews_per_month                 785
calculated_host_listings_count    922
availability_365                  922
Name: Mission, dtype: int64

In [26]:
# let's make a function since we might want to ask the same question for other columns
def most(df, column):
    return df.groupby(column).count().sort_values('id', ascending=False).iloc[0]

In [27]:
# Host with the most has 38 listings!
most(df, 'host_id')

id                                38
name                              38
host_name                         34
neighbourhood_group                0
neighbourhood                     38
latitude                          38
longitude                         38
room_type                         38
price                             38
minimum_nights                    38
number_of_reviews                 38
last_review                       22
reviews_per_month                 22
calculated_host_listings_count    38
availability_365                  38
Name: 6346492, dtype: int64

In [28]:
# Presidio is the most expensive neighborhood
df.groupby('neighbourhood')['price'].median().sort_values(ascending=False)

neighbourhood
Presidio                 265.0
Marina                   229.0
Seacliff                 224.0
Pacific Heights          210.0
Russian Hill             207.5
North Beach              195.0
Presidio Heights         194.5
Financial District       190.0
Potrero Hill             180.0
Castro/Upper Market      180.0
Noe Valley               180.0
Nob Hill                 179.0
South of Market          175.0
Glen Park                175.0
Haight Ashbury           175.0
Downtown/Civic Center    169.0
Golden Gate Park         168.0
Twin Peaks               167.0
Western Addition         165.0
Diamond Heights          156.5
Mission                  150.0
Bernal Heights           150.0
Chinatown                150.0
Inner Sunset             150.0
Inner Richmond           150.0
Outer Richmond           142.5
West of Twin Peaks       130.0
Outer Mission            129.0
Bayview                  107.5
Crocker Amazon           105.0
Lakeshore                105.0
Parkside                 

In [29]:
# looks like Kepa tops the charts... and has possibly 3 identical listings
df.sort_values('number_of_reviews', ascending=False).head(20)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
4718,33577,Potrero Hill Garden Suite,145258,Kepa,NaN,Potrero Hill,37.765146,-122.397501,Entire home/apt,190,1,394,2015-10-30,7.04,4,324
2776,8739,"Mission Sunshine, with Private Bath",7149,Ivan & Wendy,NaN,Mission,37.755592,-122.423657,Private room,120,1,352,2015-10-20,4.62,1,320
4700,205842,Potrero Hill Garden Cabana,145258,Kepa,NaN,Potrero Hill,37.764874,-122.397627,Entire home/apt,190,1,349,2015-10-29,8.73,4,316
4719,33578,Potrero Hill Garden Lookout,145258,Kepa,NaN,Potrero Hill,37.765256,-122.397305,Entire home/apt,190,1,328,2015-10-27,5.26,4,312
4752,256148,Spacious Bed/Bath Suite in Castro,1239523,Chris,NaN,Castro/Upper Market,37.758759,-122.434729,Private room,139,1,278,2015-10-28,5.76,1,323
5062,12522,Perfectly located Castro,18431,Brenda,NaN,Castro/Upper Market,37.760722,-122.428153,Private room,69,3,273,2015-10-30,3.87,1,15
2449,748222,Sunny Room Heart of the Mission!,516404,Michael And Yesica,NaN,Mission,37.756212,-122.415546,Private room,100,1,268,2015-10-29,7.40,2,264
2447,748256,Valencia Corridor Center Of It All!,516404,Michael And Yesica,NaN,Mission,37.754954,-122.416831,Private room,100,1,265,2015-10-28,7.32,2,310
1589,545685,Garden Suite Private Bathrm GGPark,2676602,Su,NaN,Outer Richmond,37.776190,-122.482308,Private room,99,1,251,2015-10-30,6.16,2,321
4615,683190,Spacious - 3000 sq. ft. SF LOFT,1545166,John,NaN,Potrero Hill,37.755311,-122.385177,Entire home/apt,250,1,246,2015-10-27,6.45,1,325


### Prof F impact

What impact would a Prof F like policy have on your city? 
> Prop F would have removed all listings that are the Entire Home and available for more than 75 days per year

In [30]:
prop_f = df[(df.room_type == 'Entire home/apt') & (df.availability_365 > 75)]
print "Prof F would have removed {0} listings of {1} total listings. This is {2} of the listings for SF".format( \
       len(prop_f), len(df), len(prop_f) / float(len(df)))

Prof F would have removed 2825 listings of 7029 total listings. This is 0.401906387822 of the listings for SF


### Prof F Revenue Lost

What would AirBnB’s revenue loss have been from a passing of Prop F? Calculate revenue from the reviews_per_month, availability_365, and price for each listing.

In [31]:
'''
We of course have to make some assumptions about the data. The documentation on the Inside AirBnB site
can give us a lot of helpful context however: http://insideairbnb.com/about.html

Assumptions:
    * 6-12% Guest service fee and 3% host service fee. 9% - 15% in total 
    * A review rate of 72% is unverifiable - therefore 50% was chosen as it sits almost exactly between 72% and 30.5%.
'''

def revenue(row):
    # availability per month... also Python 2.7 does integer division
    availability_per_month = row.availability_365 / float(30)
    
    # Only 50% of people who stay write reviews
    stays_per_month = row.reviews_per_month * 2
    
    revenue_per_month = stays_per_month * availability_per_month * row.price
    
    return revenue_per_month

New Python string formatting: https://pyformat.info/

In [32]:
revenues_per_month = df.apply(revenue, axis=1)

message = '''
If Prop F passed, individual 'high occupancy' AirBnB host would have lost ${0:.2f}/month and collectively 
all the high occupancy hosts in the city would have lost approximately ${1:.3f} million/month.
'''

print message.format(revenues_per_month.median(), revenues_per_month.sum() / 1000000)


If Prop F passed, individual 'high occupancy' AirBnB host would have lost $2334.93/month and collectively 
all the high occupancy hosts in the city would have lost approximately $28.337 million/month.



In [33]:
# the most conservative estimate using the lowest guest service fee rate (6%).
# 6% (guest fee) + 3% (host fee) = 9% (airbnb revenue)
million_per_month = (revenues_per_month.sum() * 0.09) / float(1000000)
"This translates into ${0:.2f} million/month of lost revenue for AirBnB".format(million_per_month)

'This translates into $2.55 million/month of lost revenue for AirBnB'

In [34]:
context = '''In the grand scheme of things this is actually a non-trivial amount of revenue for AirBnB... 
Sources report that they make ~$900 million of revenue in 2015 (which equals $75 million/month).

And the SF revenue lost to Prop F would be {0}% of their total revenue.'''

print context.format((2.55/75) * 100)

In the grand scheme of things this is actually a non-trivial amount of revenue for AirBnB... 
Sources report that they make ~$900 million of revenue in 2015 (which equals $75 million/month).

And the SF revenue lost to Prop F would be 3.4% of their total revenue.


https://www.quora.com/How-much-revenue-is-Airbnb-making

https://www.airbnb.com/help/article/104/what-are-guest-service-fees

## Extra: `pandas` advanced queries

In [35]:
median_nights = df.groupby('host_id')['availability_365'].median()
median_df = pd.DataFrame(median_nights).reset_index()
median_df.columns = ['host_id', 'median_listing_availability']

# Some hosts would circumvent Prop F by renting out all of the rooms of a apartment, but list them as
# shared rooms. This is more like a hostel.
df.merge(median_df, on='host_id') \
  .groupby('host_id').first() \
  .sort_values('calculated_host_listings_count', ascending=False) \
  .head(20)

,id,name,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,median_listing_availability
host_id,,,,,,,,,,,,,,,,
6346492,7958349,Downtown Top U. Sq. Hotel/TS Studio,Dana,NaN,Downtown/Civic Center,37.788465,-122.409592,Entire home/apt,180,1,0,2015-10-14,0.93,34,36,18.0
7206100,6742488,Looky Home Moscone,NaN,NaN,Financial District,37.782433,-122.401098,Shared room,65,30,4,2015-10-05,2.35,31,317,365.0
38836,10828,Haight - Buena Vista Park 1BR,Bernat,NaN,Haight Ashbury,37.769725,-122.438780,Entire home/apt,171,3,15,2015-09-28,0.46,28,267,267.5
23429336,7292853,3br/2ba apt by Golden Gate Park!,NaN,NaN,Outer Richmond,37.777021,-122.488989,Entire home/apt,190,30,4,2015-08-10,1.14,24,365,348.5
4581592,9186095,"SF, Golden Gate, Castro, Wine-Bunk5",Erik,NaN,Parkside,37.742180,-122.471665,Shared room,39,3,0,2015-10-14,2.73,22,365,365.0
1091811,1864687,Sunlit Cottage in Russian Hill,Sean,NaN,Russian Hill,37.798802,-122.415641,Entire home/apt,229,5,30,2014-10-29,1.24,20,0,0.0
11186281,4839456,Single bed in a spacious loft,Francesca & Team,NaN,South of Market,37.780413,-122.406470,Shared room,70,7,2,2015-06-13,0.40,17,257,364.0
305973,687430,Mission Outr CozyBedRm3 + MinFridge,NaN,NaN,Excelsior,37.729604,-122.427285,Private room,40,30,12,2015-08-23,0.33,16,318,319.0
1682188,7762892,Hacknsleep(com)/1-roomfor2,Zarina&Yaroslav,NaN,Parkside,37.741953,-122.486760,Shared room,45,1,4,2015-10-19,1.85,16,365,365.0


In [36]:
many_listings = df.groupby('host_id').count().id > 3

In [37]:
aggregations = {'id':'count', 'price':'median', 'minimum_nights': 'median', 'availability_365': 'median'}
df[df.host_id.isin(many_listings.index)].groupby(['host_id', 'room_type']).agg(aggregations)

,,minimum_nights,price,availability_365,id
host_id,room_type,,,,
46,Entire home/apt,2.0,200.0,334.0,1
240,Entire home/apt,1.0,223.0,311.0,1
322,Shared room,1.0,155.0,178.0,1
878,Entire home/apt,3.0,300.0,365.0,1
1169,Entire home/apt,2.0,170.0,314.0,1
3163,Private room,1.0,118.5,330.5,2
3363,Entire home/apt,30.0,170.0,223.0,1
5974,Private room,17.0,82.5,34.0,2
6267,Entire home/apt,3.0,250.0,94.0,1
